In [1]:
import os, json
from pathlib import Path, PureWindowsPath # for Unix compatibility

from cri.robot import SyncRobot
from cri.controller import Mg400Controller as Controller
from vsp.video_stream import CvImageOutputFileSeq, CvVideoDisplay, CvPreprocVideoCamera   
from vsp.processor import CameraStreamProcessor, AsyncProcessor

from tactile_servoing_2d.lib.common import home, control
from pose_models_2d.lib.models.cnn_model import CNNmodel as Model

data_path = os.path.join(os.environ["DATAPATH"], "open", "tactile-servoing-2d-dobot")

In [2]:
# User-defined paths
test_dir = os.path.join("digitac", "servo_surface2d", "flower") 
model_dir = os.path.join("digitac", "model_surface2d", "train", "train2d_cnn")  

# Open saved meta dictionaries
with open(os.path.join(data_path, model_dir, "meta.json"), 'r') as f: 
    model_meta = json.load(f)

# Make the new meta dictionary
meta = {**model_meta, 
    # ~~~~~~~~~ Paths ~~~~~~~~~#
    "meta_file": os.path.join(test_dir, "meta.json"),        
    "test_image_dir": os.path.join(test_dir, "frames_bw"), # to store tactile images
    "test_df_file": os.path.join(test_dir, "predictions.csv"), # to store predictions and robot poses
    # ~~~~~~~~~ Robot movements ~~~~~~~~~#
    "home_pose": [330, -100, -100, 0, 0, 0],  
    "linear_speed": 10,
    "num_steps": 400, # preset number of servo control steps
    # ~~ disk ~~
    # "work_frame": [330, -89+38, -140, 0, 0, 12], # tactip 
    # "work_frame": [330, -78+24, -146, 0, 0, -8], # digitac
    # ~~ flower (top toward robot) ~~
    # "work_frame": [330, -92+38, -140, 0, 0, 12], # tactip
    "work_frame": [330, -81+24, -146, 0, 0, -8], # digitac
    # ~~ square ~~
    # "work_frame": [330, -84+38, -140, 0, 0, 12], # tactip
    # "work_frame": [330, -73+24, -146, 0, 0, -8], # digitac
    # ~~ other settings ~~        
    "source": 0, # set point of controller (also used to make fixed steps)
    "r": [1, 0, 0, 0, 0, 0], # note: pose_6 5 for digitac on square # set point of contrller (value 1 used to make fixed steps) 
    "kp": [1, 0.5, 0, 0, 0, 1], # p-control gains 
    "ki": [0, 0.3, 0, 0, 0, 0.1], # i-control gains
    "ei_bnd": [[0, 5, 0, 0, 0, 45], [0, -5, 0, 0, 0, -45]], # windup bounds 
    # ~~~~~~~~~ Comments ~~~~~~~~~#
    "comments": "servo control with 2d surface on MG400"
    }

In [3]:
# Save dictionary to file
os.makedirs(os.path.join(data_path, test_dir), exist_ok=True)
with open(os.path.join(data_path, meta["meta_file"]), 'w') as f:
    json.dump(meta, f)

# Absolute posix paths
for key in [k for k in meta.keys() if "file" in k or "dir" in k]:
    meta[key] = os.path.join(data_path, meta[key])
    meta[key] = Path(PureWindowsPath(meta[key])).as_posix() # for Unix

In [4]:
def make_sensor(size, crop, threshold, exposure, source, **kwargs):  # amcap: reset all settings; autoexposure off; saturdation max
    camera = CvPreprocVideoCamera(size, crop, threshold, exposure=exposure, source=source)
    for _ in range(5): camera.read() # Hack - camera transient   
    return AsyncProcessor(CameraStreamProcessor(camera=camera,
            display=CvVideoDisplay(name='sensor'),
            writer=CvImageOutputFileSeq()))

In [5]:
# Startup/load model 
model = Model()
model.load_model(**meta)

# Control robot with loaded model
with SyncRobot(Controller()) as robot, make_sensor(**meta) as sensor:       
    home(robot, **meta)     
    pred, _ = control(robot, sensor, model, **meta)  

# Save run to file
pred.to_csv(meta["test_df_file"], index=False)

Client connected to Dobot MG400...


(MainThread) Model was constructed with shape (None, 160, 120, 1) for input Tensor("input:0", shape=(None, 160, 120, 1), dtype=float32), but it was called on an input with incompatible shape (None, 120, 160, 1).


1/1 [==============================] - 0s 999us/step
1: v=[ 1  0 -0.5 -0  0 -0]
     y=[ 0  -0.3   0   0   0  -6.32]
joints=[-13.87  67.02  67.28   5.87]
1/1 [==============================] - 0s 1ms/step
2: v=[ 1.99  0.33 -0.5  -0   0   6.95]
     y=[ 0  -0.51  0   0   0  -5.13]
joints=[-13.72  67.52  66.14  12.59]
1/1 [==============================] - 0s 998us/step
3: v=[ 2.91  1.05 -0.5  -0   0  13.23]
     y=[ 0  -0.4   0   0   0  -1.59]
joints=[-13.47  67.98  65.11  18.7 ]
1/1 [==============================] - 0s 1ms/step
4: v=[ 3.74  1.9  -0.5  -0   0  16.13]
     y=[ 0  -0.34  0   0   0  -3.25]
joints=[-13.36  68.11  64.83  19.98]
1/1 [==============================] - 0s 1ms/step
5: v=[ 4.49  2.89 -0.5  -0   0  21.02]
     y=[ 0  -0.09  0   0   0  -2.87]
joints=[-12.93  68.61  63.76  25.94]
1/1 [==============================] - 0s 0s/step
6: v=[ 5.18  3.87 -0.5  -0   0  25.81]
     y=[ 0   0.09  0   0   0  -3.45]
joints=[-12.61  68.96  63.03  30.42]
1/1 [====================